In [ ]:
from csc4585 import *

In [ ]:
%%java
public class Jav0 {
    static int i=0;
    public static void main(String[] args) {
        for(int k=0;k<1000;k++) {
            // instantiate and start a new java thread
            new Thread() {
                public void run() {
                    // update of static variable i is
                    // attempted by several threads at
                    // once, causing a race condition.
                    for(int j=0;j<1000;j++)
                        i++;
                    System.out.println("i="+i);
                }
            }.start();
        }
    }
}

In [ ]:
%%java
public class Jav1 {
    static int i=0;
    public static void main(String[] args) {
        for(int k=0;k<1000;k++) {
            Thread t = new Thread() {
                public void run() {
                    for(int j=0;j<1000;j++)
                        i++;
                    System.out.println("i="+i);
                }
            };
            t.start();
            
            // Here, we wait for each thread to
            // finish. By default, the Java program
            // will keep running until all threads
            // finish anyway. Since we launch each
            // thread then wait for it to finish,
            // we're essentially running a sequential
            // program instead of a parallel one.
            try {
                t.join();
            } catch(InterruptedException ie) {
                return;
            }
        }
    }
}

In [ ]:
%%java
public class Jav2 {
    static int i=0;
    public static void main(String[] args) {
        for(int k=0;k<1000;k++) {
            Thread t = new Thread() {
                public void run() {
                    for(int j=0;j<1000;j++)
                        i++;
                    System.out.println("i="+i);
                }
            };
            // Setting the Daemon state to true
            // means that Java is allowed to terminate
            // before this thread finishes.
            t.setDaemon(true);
            t.start();
        }
    }
}

In [ ]:
%%java
public class Jav5 {
  static int i=0;
  public static void main(String[] args) {
    // Create an array of threads...
    Thread[] p = new Thread[100];
      
    // Fill in the array and start
    // each thread
    for(int j=0;j<p.length;j++) {
      p[j] = new Thread() {
        public void run() {
          for(int j=0;j<1000;j++)
            i++;
          System.out.println("i="+i);
        }
      };
      p[j].start();
    }
      
    // Join each thread in the array
    for(int j=0;j<p.length;j++) {
      try {
        p[j].join();
      } catch(InterruptedException ie) {
        return;
      }
    }
  }
}

In [ ]:
%%java
public class Jav6 {
  static int i=0;
  public static void main(String[] args) {
    Thread[] p = new Thread[100];
    for(int j=0;j<p.length;j++) {
      p[j] = new Thread() {
        public void run() {
          for(int j=0;j<1000;j++)
            // Synchronizing on the class
            // ensures that no two threads
            // update the static variable
            // at the same time.
            synchronized(Jav6.class) {
              i++;
            }
          System.out.println("i="+i);
        }
      };
      p[j].start();
    }
    for(int j=0;j<p.length;j++) {
      try {
        p[j].join();
      } catch(InterruptedException ie) {
        return;
      }
    }
  }
}

In [ ]:
%%java
public class Jav7 {
  static int i=0;
  public static void main(String[] args) {
    Thread[] p = new Thread[100];
    for(int j=0;j<p.length;j++) {
      p[j] = new Thread() {
        public void run() {
          for(int j=0;j<1000;j++)
            synchronized(Jav7.class) {
              i++;
            }
          System.out.println("i="+i);
        }
      };
      p[j].start();
    }
    for(int j=0;j<p.length;j++) {
      try {
        p[j].join();
      } catch(InterruptedException ie) {
        return;
      }
    }
    // Add an AT FINISH message
    System.out.println("AT FINISH: "+i);
  }
}

In [ ]:
%%java
public class Jav8 {
    static int i=0;
    
    // Move constants to final static variables
    static final int THREADS = 100;
    static final int INCRS_PER_THREAD = 1000;
    
    public static void main(String[] args) {
        // This special thread watches the variable
        // i and waits to see when the updates are
        // finished.
        Thread fin = new Thread() {
            public void run() {
                synchronized(Jav8.class) {
                    while(i < THREADS*INCRS_PER_THREAD) {
                        try {
                            Jav8.class.wait();
                        } catch(InterruptedException ie) {
                            return;
                        }
                    }
                }
                System.out.println("AT FINISH: "+i);
            }
        };
        fin.start();
        Thread[] p = new Thread[THREADS];
        for(int j=0;j<p.length;j++) {
            p[j] = new Thread() {
                public void run() {
                    synchronized(Jav8.class) {
                        for(int j=0;j<INCRS_PER_THREAD;j++)
                            i++;
                        Jav8.class.notify();
                        System.out.println("i="+i);
                    }
                }
            };
            p[j].start();
        }
        for(int j=0;j<p.length;j++) {
            try {
                p[j].join();
            } catch(InterruptedException ie) {
                return;
            }
        }
        try {
            fin.join();
        } catch(InterruptedException ie) {
            return;
        }
    }
}

In [ ]:
%%java
public class Jav9 {
    static int i=0;
    static final int THREADS = 100;
    static final int INCRS_PER_THREAD = 1000;
    public static void main(String[] args) {
        Thread fin = new Thread() {
            public void run() {
                synchronized(Jav9.class) {
                    while(i < THREADS*INCRS_PER_THREAD) {
                        try {
                            Jav9.class.wait();
                        } catch(InterruptedException ie) {
                            return;
                        }
                    }
                }
                System.out.println("AT FINISH: "+i);
            }
        };
        fin.start();
        Thread[] p = new Thread[THREADS];
        for(int j=0;j<p.length;j++) {
            p[j] = new Thread() {
                public void run() {
                    synchronized(Jav9.class) {
                        // Move more work inside the synchronized block
                        for(int j=0;j<INCRS_PER_THREAD;j++)
                            i++;
                        if(i==THREADS*INCRS_PER_THREAD)
                            Jav9.class.notify();
                        System.out.println("i="+i);
                    }
                }
            };
            p[j].start();
        }
        try {
            fin.join();
        } catch(InterruptedException ie) {
            return;
        }
    }
}